In [1]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [6]:
df1 = pd.read_csv('df_dropped_values.csv')
df2 = pd.read_csv('df_imputated_values.csv')

In [12]:
df1.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('bool')], dtype=object)

In [13]:
df2.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('bool')], dtype=object)

In [14]:
df1 = df1.astype({col: int for col in df1.select_dtypes('bool').columns})

In [15]:
df2 = df2.astype({col: int for col in df2.select_dtypes('bool').columns})